In [1]:
import nltk
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer

text1 = "Python is a 2000 made-for-TV horror movie directed by Richard \
Clabaugh. The film features several cult favorite actors, including William \
Zabka of The Karate Kid fame, Wil Wheaton, Casper Van Dien, Jenny McCarthy, \
Keith Coogan, Robert Englund (best known for his role as Freddy Krueger in the \
A Nightmare on Elm Street series of films), Dana Barron, David Bowe, and Sean \
Whalen. The film concerns a genetically engineered snake, a python, that \
escapes and unleashes itself on a small town. It includes the classic final\
girl scenario evident in films like Friday the 13th. It was filmed in Los Angeles, \
California and Malibu, California. Python was followed by two sequels: Python \
II (2002) and Boa vs. Python (2004), both also made-for-TV films."

text2 = "Python, from the Greek word (πύθων/πύθωνας), is a genus of \
nonvenomous pythons[2] found in Africa and Asia. Currently, 7 species are \
recognised.[2] A member of this genus, P. reticulatus, is among the longest \
snakes known."

text3 = "The Colt Python is a .357 Magnum caliber revolver formerly \
manufactured by Colt's Manufacturing Company of Hartford, Connecticut. \
It is sometimes referred to as a \"Combat Magnum\".[1] It was first introduced \
in 1955, the same year as Smith & Wesson's M29 .44 Magnum. The now discontinued \
Colt Python targeted the premium revolver market segment. Some firearm \
collectors and writers such as Jeff Cooper, Ian V. Hogg, Chuck Hawks, Leroy \
Thompson, Renee Smeets and Martin Dougherty have described the Python as the \
finest production revolver ever made."

In [3]:
#斷詞方法,並移除標點符號
def get_tokens(text):
    lowers = text.lower()
#remove the punctuation using the character deletion step of translate
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lowers.translate(remove_punctuation_map)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

In [19]:
tokens = get_tokens(text1)
print("#斷詞結果: \n", tokens , "\n")

#計數器
count = Counter(tokens)
print ("#詞頻: \n", count.most_common(10) , "\n")


#斷詞結果: 
 ['python', 'is', 'a', '2000', 'madefortv', 'horror', 'movie', 'directed', 'by', 'richard', 'clabaugh', 'the', 'film', 'features', 'several', 'cult', 'favorite', 'actors', 'including', 'william', 'zabka', 'of', 'the', 'karate', 'kid', 'fame', 'wil', 'wheaton', 'casper', 'van', 'dien', 'jenny', 'mccarthy', 'keith', 'coogan', 'robert', 'englund', 'best', 'known', 'for', 'his', 'role', 'as', 'freddy', 'krueger', 'in', 'the', 'a', 'nightmare', 'on', 'elm', 'street', 'series', 'of', 'films', 'dana', 'barron', 'david', 'bowe', 'and', 'sean', 'whalen', 'the', 'film', 'concerns', 'a', 'genetically', 'engineered', 'snake', 'a', 'python', 'that', 'escapes', 'and', 'unleashes', 'itself', 'on', 'a', 'small', 'town', 'it', 'includes', 'the', 'classic', 'finalgirl', 'scenario', 'evident', 'in', 'films', 'like', 'friday', 'the', '13th', 'it', 'was', 'filmed', 'in', 'los', 'angeles', 'california', 'and', 'malibu', 'california', 'python', 'was', 'followed', 'by', 'two', 'sequels', 'python', 'ii

In [14]:
#詞幹還原方法
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [18]:
tokens = get_tokens(text1)

#移除停用詞
filtered = [w for w in tokens if not w in stopwords.words('english')]

#計數器
count = Counter(filtered)
print("結果顯示: \n", count)

結果顯示: 
 Counter({'python': 5, 'films': 3, 'madefortv': 2, 'film': 2, 'california': 2, '2000': 1, 'horror': 1, 'movie': 1, 'directed': 1, 'richard': 1, 'clabaugh': 1, 'features': 1, 'several': 1, 'cult': 1, 'favorite': 1, 'actors': 1, 'including': 1, 'william': 1, 'zabka': 1, 'karate': 1, 'kid': 1, 'fame': 1, 'wil': 1, 'wheaton': 1, 'casper': 1, 'van': 1, 'dien': 1, 'jenny': 1, 'mccarthy': 1, 'keith': 1, 'coogan': 1, 'robert': 1, 'englund': 1, 'best': 1, 'known': 1, 'role': 1, 'freddy': 1, 'krueger': 1, 'nightmare': 1, 'elm': 1, 'street': 1, 'series': 1, 'dana': 1, 'barron': 1, 'david': 1, 'bowe': 1, 'sean': 1, 'whalen': 1, 'concerns': 1, 'genetically': 1, 'engineered': 1, 'snake': 1, 'escapes': 1, 'unleashes': 1, 'small': 1, 'town': 1, 'includes': 1, 'classic': 1, 'finalgirl': 1, 'scenario': 1, 'evident': 1, 'like': 1, 'friday': 1, '13th': 1, 'filmed': 1, 'los': 1, 'angeles': 1, 'malibu': 1, 'followed': 1, 'two': 1, 'sequels': 1, 'ii': 1, '2002': 1, 'boa': 1, 'vs': 1, '2004': 1, 'also'

In [20]:
#有些字詞時態；單複數不同，但詞義相同 -->詞幹還原
tokens = get_tokens(text1)
filtered = [w for w in tokens if not w in stopwords.words('english')]
stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)

In [23]:
def tf(word, count):
    return count[word] / sum(count.values())
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)
def idf(word, count_list):
    return math.log(len(count_list) / (n_containing(word, count_list)))
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

In [32]:
countlist = [count1, count2, count3]
for i, count in enumerate(countlist):
    print("Top words in document {}".format(i))
    scores = {word: tfidf(word, count, countlist) for word in count}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

NameError: name 'count1' is not defined